## 需求

现在我的需求是要获取图中多个成功的buildid信息，哪种是成功的buildid呢？就是有标记`icon-blue`的。

![](./buildid_rquirement.png)

## 使用`requests`

首先尝试的是使用`requests`，但是通过它获取到的网页信息太有限了（内容见requests_output.txt），这是因为该网页包含了太多的javascript脚本，这些脚本使得只有在真正的浏览该网页时一些必要的信息才会加载。

In [4]:
import requests

url = 'https://ece-ci.dynamic.nsn-net.net/job/MASTER/job/GNB/job/UPLANE/job/L2-LO/job/SCT.fuse.asib_abio/buildTimeTrend'
r = requests.get(url, verify=False)

with open("requests_output.txt", "w", encoding="utf-8") as f:
    f.write(r.text)

c:\Users\lianbche\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ece-ci.dynamic.nsn-net.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## 使用`selenium`

这个时候我们就需要试试专门用来默认真人访问网页的`selenium`函数库。可以看到，即便相同的url，通过requests访问得到的结果仅有64kb，但通过selenium访问得到的结果有1542kb，相差24倍之多。

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time


profile = webdriver.FirefoxProfile()
settings = {
    'network.proxy.type': 1,  # 0: 不使用代理；1: 手动配置代理
    'network.proxy.http': '10.144.1.10',
    'network.proxy.http_port': 8080,
    'network.proxy.ssl': '10.144.1.10',  # https的网站,
    'network.proxy.ssl_port': 8080,
}

for key, value in settings.items():
    profile.set_preference(key, value)
profile.update_preferences()

url = 'https://ece-ci.dynamic.nsn-net.net/job/MASTER/job/GNB/job/UPLANE/job/L2-LO/job/SCT.fuse.asib_abio/buildTimeTrend'
driver = webdriver.Firefox(firefox_profile=profile)
driver.get(url)
time.sleep(15)

with open("selenium_pagesource.txt", "w", encoding="utf-8") as f:
    f.write(driver.page_source)

bs = BeautifulSoup(driver.page_source)
succ_builds = bs.find_all(attrs={"data":"4"}, limit=2)
for build in succ_builds:
    print(build.a['href'].split('/')[0])
driver.quit()


38371/console
38370/console


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=61568): Max retries exceeded with url: /session/596b297b-1857-41b6-b151-bf31d93a1d95/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A85DBB6610>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))